In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from Utils.data_get_utils import get_duka_data_df
duka_5m_2020_df = get_duka_data_df("../data/EURUSD5_duka_2020.csv", "EURUSD")
duka_5m_2021_df = get_duka_data_df("../data/EURUSD5_duka_2021.csv", "EURUSD")
duka_5m_df = pd.concat([duka_5m_2020_df, duka_5m_2021_df])

duka_5m_df.head()

,symbol,datetime,open,high,low,close,volume
264,EURUSD,2020-01-01 22:00:00,1.12136,1.12139,1.12120,1.12125,47.60
265,EURUSD,2020-01-01 22:05:00,1.12125,1.12127,1.12125,1.12127,91.79
266,EURUSD,2020-01-01 22:10:00,1.12127,1.12127,1.12123,1.12127,64.98
267,EURUSD,2020-01-01 22:15:00,1.12127,1.12154,1.12127,1.12146,85.07
268,EURUSD,2020-01-01 22:20:00,1.12144,1.12153,1.12137,1.12146,123.93


In [3]:
from Utils.data_get_utils import get_fm_rc_data_df
from Utils.data_get_utils import calc_profit

rc_df = get_fm_rc_data_df("../data/yht_rc.csv")
# current research is based on EURUSD
rc_df=rc_df[rc_df["symbol"]=="EURUSD"].reset_index(drop=True)

In [4]:
rc_df.head()

,symbol,action,lots,open_date,close_date,open_price,close_price,profits,pips,commission,swaps,profits_calc,commission_by_pips,date_var,swap_per_date_var_and_pip
0,EURUSD,buy,0.01,2021-07-09 02:05:44,2021-07-09 03:08:27,1.18363,1.18375,0.12,1.2,-0.04,0.0,-0.028,-0.4,0,0.0
1,EURUSD,buy,0.01,2021-07-08 15:41:46,2021-07-08 15:52:30,1.18410,1.18440,0.30,3.0,-0.04,0.0,-0.010,-0.4,0,0.0
2,EURUSD,sell,0.01,2021-07-08 13:55:29,2021-07-08 14:59:42,1.18600,1.18538,0.62,6.2,-0.05,0.0,0.012,-0.5,0,0.0
3,EURUSD,sell,0.01,2021-07-08 14:29:13,2021-07-08 14:48:43,1.18640,1.18613,0.27,2.7,-0.05,0.0,-0.023,-0.5,0,0.0
4,EURUSD,sell,0.01,2021-07-08 13:32:00,2021-07-08 13:46:35,1.18589,1.18543,0.46,4.6,-0.05,0.0,-0.004,-0.5,0,0.0


In [5]:
rc_df.columns

Index(['symbol', 'action', 'lots', 'open_date', 'close_date', 'open_price',
       'close_price', 'profits', 'pips', 'commission', 'swaps', 'profits_calc',
       'commission_by_pips', 'date_var', 'swap_per_date_var_and_pip'],
      dtype='object')

In [6]:
from Utils.data_process_utils import get_period_bar_date
rc_df = get_period_bar_date(rc_df, ["open_date","close_date"], 5)
rc_df.head()

,symbol,action,lots,open_date,close_date,open_price,close_price,profits,pips,commission,swaps,profits_calc,commission_by_pips,date_var,swap_per_date_var_and_pip,open_date_mod_5,close_date_mod_5
0,EURUSD,buy,0.01,2021-07-09 02:05:44,2021-07-09 03:08:27,1.18363,1.18375,0.12,1.2,-0.04,0.0,-0.028,-0.4,0,0.0,2021-07-09 02:05:00,2021-07-09 03:05:00
1,EURUSD,buy,0.01,2021-07-08 15:41:46,2021-07-08 15:52:30,1.18410,1.18440,0.30,3.0,-0.04,0.0,-0.010,-0.4,0,0.0,2021-07-08 15:40:00,2021-07-08 15:50:00
2,EURUSD,sell,0.01,2021-07-08 13:55:29,2021-07-08 14:59:42,1.18600,1.18538,0.62,6.2,-0.05,0.0,0.012,-0.5,0,0.0,2021-07-08 13:55:00,2021-07-08 14:55:00
3,EURUSD,sell,0.01,2021-07-08 14:29:13,2021-07-08 14:48:43,1.18640,1.18613,0.27,2.7,-0.05,0.0,-0.023,-0.5,0,0.0,2021-07-08 14:25:00,2021-07-08 14:45:00
4,EURUSD,sell,0.01,2021-07-08 13:32:00,2021-07-08 13:46:35,1.18589,1.18543,0.46,4.6,-0.05,0.0,-0.004,-0.5,0,0.0,2021-07-08 13:30:00,2021-07-08 13:45:00


### 检查是否交易记录是否在输入的EURUSD数据里面

In [7]:
%%time
# rc_df中的时间必须都在duka_5m_df当中存在
duka_5m_outer_join_rc_df = pd.merge(duka_5m_df, rc_df, left_on="datetime", right_on="open_date_mod_5", how="outer")
duka_5m_inner_join_rc_df = pd.merge(duka_5m_df, rc_df, left_on="datetime", right_on="open_date_mod_5", how="inner")
duka_5m_right_join_rc_df = pd.merge(duka_5m_df, rc_df, left_on="datetime", right_on="open_date_mod_5", how="right")
duka_5m_left_join_rc_df = pd.merge(duka_5m_df, rc_df, left_on="datetime", right_on="open_date_mod_5", how="left")
# duka_5m_left_join_rc_df = duka_5m_left_join_rc_df.dropna()

CPU times: user 40.4 ms, sys: 8.07 ms, total: 48.5 ms
Wall time: 48.3 ms


In [8]:
# classical methods to define the assert
assert len(duka_5m_left_join_rc_df.dropna()) == len(rc_df), (
        f'Some of the followme records are not in the instrument EURUSD input data.')
print(len(duka_5m_left_join_rc_df.dropna()))  
len(rc_df)

1890


1890

### 把时间周期关系全部搞清出来

In [9]:
# gmt + 8 == peking 
# gmt + 3 == tickmill

### 交易记录复现dataframe

In [10]:
print("bar data length:", len(duka_5m_df))

bar data length: 114042


In [11]:
duka_5m_df.head()

,symbol,datetime,open,high,low,close,volume
264,EURUSD,2020-01-01 22:00:00,1.12136,1.12139,1.12120,1.12125,47.60
265,EURUSD,2020-01-01 22:05:00,1.12125,1.12127,1.12125,1.12127,91.79
266,EURUSD,2020-01-01 22:10:00,1.12127,1.12127,1.12123,1.12127,64.98
267,EURUSD,2020-01-01 22:15:00,1.12127,1.12154,1.12127,1.12146,85.07
268,EURUSD,2020-01-01 22:20:00,1.12144,1.12153,1.12137,1.12146,123.93


In [12]:
rc_df.head()

,symbol,action,lots,open_date,close_date,open_price,close_price,profits,pips,commission,swaps,profits_calc,commission_by_pips,date_var,swap_per_date_var_and_pip,open_date_mod_5,close_date_mod_5
0,EURUSD,buy,0.01,2021-07-09 02:05:44,2021-07-09 03:08:27,1.18363,1.18375,0.12,1.2,-0.04,0.0,-0.028,-0.4,0,0.0,2021-07-09 02:05:00,2021-07-09 03:05:00
1,EURUSD,buy,0.01,2021-07-08 15:41:46,2021-07-08 15:52:30,1.18410,1.18440,0.30,3.0,-0.04,0.0,-0.010,-0.4,0,0.0,2021-07-08 15:40:00,2021-07-08 15:50:00
2,EURUSD,sell,0.01,2021-07-08 13:55:29,2021-07-08 14:59:42,1.18600,1.18538,0.62,6.2,-0.05,0.0,0.012,-0.5,0,0.0,2021-07-08 13:55:00,2021-07-08 14:55:00
3,EURUSD,sell,0.01,2021-07-08 14:29:13,2021-07-08 14:48:43,1.18640,1.18613,0.27,2.7,-0.05,0.0,-0.023,-0.5,0,0.0,2021-07-08 14:25:00,2021-07-08 14:45:00
4,EURUSD,sell,0.01,2021-07-08 13:32:00,2021-07-08 13:46:35,1.18589,1.18543,0.46,4.6,-0.05,0.0,-0.004,-0.5,0,0.0,2021-07-08 13:30:00,2021-07-08 13:45:00


In [13]:
duka_5m_left_join_rc_df = pd.merge(duka_5m_df, rc_df, 
                                   left_on="datetime", right_on="open_date_mod_5", 
                                   how="left")

### join 后得到的结果数量比原有的left_df: duka_5m_df 多很正常
### 因为同一个位置有可能有多个单子

In [14]:
# 保证所有followme当中出现的订单都在左表: duka_5m_df 中出现过
assert len(rc_df) == len(duka_5m_left_join_rc_df[duka_5m_left_join_rc_df["symbol_y"].notnull()]),(
    f'Some of the records data(rc_df) is lost, rc: ', 
    len(rc_df), " results: ", 
    len(duka_5m_left_join_rc_df[duka_5m_left_join_rc_df["symbol_y"].notnull()])
)

In [15]:
%%time
from Utils.data_process_utils import slice_df_by_timerange
from datetime import timedelta

rc_start_dt = duka_5m_left_join_rc_df.dropna()["open_date"].min() - timedelta(hours=2)
rc_end_dt = duka_5m_left_join_rc_df.dropna()["close_date"].max() + timedelta(hours=2)
print("slicing date range : (", rc_start_dt, ", ", rc_end_dt, ")")
duka_5m_left_join_rc_date_filtered_df = slice_df_by_timerange(duka_5m_left_join_rc_df, "datetime", rc_start_dt, rc_end_dt)
analysis_df = duka_5m_left_join_rc_date_filtered_df.reset_index(drop=True)
analysis_df[analysis_df["symbol_y"].notnull()][["symbol_y","open_date_mod_5","close_date_mod_5"]].head()

slicing date range : ( 2020-10-14 08:08:18 ,  2021-07-09 05:08:27 )
CPU times: user 28.4 ms, sys: 27 µs, total: 28.5 ms
Wall time: 28 ms


,symbol_y,open_date_mod_5,close_date_mod_5
23,EURUSD,2020-10-14 10:05:00,2020-10-14 12:30:00
27,EURUSD,2020-10-14 10:25:00,2020-10-14 12:30:00
42,EURUSD,2020-10-14 11:40:00,2020-10-14 12:30:00
47,EURUSD,2020-10-14 12:05:00,2020-10-14 12:30:00
49,EURUSD,2020-10-14 12:15:00,2020-10-14 15:30:00


In [16]:
analysis_df.columns

Index(['symbol_x', 'datetime', 'open', 'high', 'low', 'close', 'volume',
       'symbol_y', 'action', 'lots', 'open_date', 'close_date', 'open_price',
       'close_price', 'profits', 'pips', 'commission', 'swaps', 'profits_calc',
       'commission_by_pips', 'date_var', 'swap_per_date_var_and_pip',
       'open_date_mod_5', 'close_date_mod_5'],
      dtype='object')

### Calculates the current holding orders base on record data 

In [17]:
%%time

def gen_dt_range(row):
    start_dt, end_dt = row["open_date_mod_5"], row["close_date_mod_5"]
    dt_range = (end_dt-start_dt)
    return dt_range

rc_df["dt_range"] = rc_df.apply(gen_dt_range, axis=1)
# Makes sure that the dt_range can be mod by 5
assert rc_df["dt_range"].apply(lambda x: x.seconds % (5*60) == 0).eq(True).all()

from datetime import datetime
from datetime import timedelta

def gen_dt_range_list(row):
    dt_range_list = []
    start_dt, end_dt = row["open_date_mod_5"], row["close_date_mod_5"]
    for dt_point in pd.date_range(start=start_dt, end=end_dt, freq='5min'):
        dt_range_list.append(dt_point.to_pydatetime())
    return dt_range_list

rc_df["dt_range_list"] = rc_df.apply(gen_dt_range_list, axis=1)
rc_df["dt_range_inter_num"] = rc_df["dt_range_list"].apply(lambda x: len(x))
rc_df["dt_range_inter_num"].sum()

CPU times: user 413 ms, sys: 4.11 ms, total: 418 ms
Wall time: 414 ms


109133

In [18]:
%%time
rc_explode_df = rc_df.explode('dt_range_list', ignore_index=True)
rc_explode_df = rc_explode_df.rename(columns={"dt_range_list": "dt_point_holding"})
# rc_explode_df.set_index(keys=["dt_point_holding"], drop=False, inplace=True)
# check_idx = 2
# print(rc_df.loc[check_idx,]["dt_range_list"])
# print(rc_df.loc[check_idx,]["open_date_mod_5"])
# print(rc_df.loc[check_idx,]["close_date_mod_5"])

CPU times: user 30.5 ms, sys: 143 µs, total: 30.6 ms
Wall time: 30.4 ms


In [19]:
print(len(rc_explode_df))
rc_explode_df.head()

109133


,symbol,action,lots,open_date,close_date,open_price,close_price,profits,pips,commission,swaps,profits_calc,commission_by_pips,date_var,swap_per_date_var_and_pip,open_date_mod_5,close_date_mod_5,dt_range,dt_point_holding,dt_range_inter_num
0,EURUSD,buy,0.01,2021-07-09 02:05:44,2021-07-09 03:08:27,1.18363,1.18375,0.12,1.2,-0.04,0.0,-0.028,-0.4,0,0.0,2021-07-09 02:05:00,2021-07-09 03:05:00,0 days 01:00:00,2021-07-09 02:05:00,13
1,EURUSD,buy,0.01,2021-07-09 02:05:44,2021-07-09 03:08:27,1.18363,1.18375,0.12,1.2,-0.04,0.0,-0.028,-0.4,0,0.0,2021-07-09 02:05:00,2021-07-09 03:05:00,0 days 01:00:00,2021-07-09 02:10:00,13
2,EURUSD,buy,0.01,2021-07-09 02:05:44,2021-07-09 03:08:27,1.18363,1.18375,0.12,1.2,-0.04,0.0,-0.028,-0.4,0,0.0,2021-07-09 02:05:00,2021-07-09 03:05:00,0 days 01:00:00,2021-07-09 02:15:00,13
3,EURUSD,buy,0.01,2021-07-09 02:05:44,2021-07-09 03:08:27,1.18363,1.18375,0.12,1.2,-0.04,0.0,-0.028,-0.4,0,0.0,2021-07-09 02:05:00,2021-07-09 03:05:00,0 days 01:00:00,2021-07-09 02:20:00,13
4,EURUSD,buy,0.01,2021-07-09 02:05:44,2021-07-09 03:08:27,1.18363,1.18375,0.12,1.2,-0.04,0.0,-0.028,-0.4,0,0.0,2021-07-09 02:05:00,2021-07-09 03:05:00,0 days 01:00:00,2021-07-09 02:25:00,13


In [20]:
%%time 
yht_rc_df_cnt = rc_explode_df.groupby(by=[rc_explode_df["dt_point_holding"]], dropna=False)\
                             .agg(cur_holding_cnt=("symbol", "count"))

CPU times: user 7.26 ms, sys: 6 µs, total: 7.26 ms
Wall time: 7.05 ms


In [21]:
df_merged_l = pd.merge(rc_explode_df, yht_rc_df_cnt, left_on="dt_point_holding", right_on="dt_point_holding", 
                       how='left')
df_merged_o = pd.merge(rc_explode_df, yht_rc_df_cnt, left_on="dt_point_holding", right_on="dt_point_holding", 
                       how='outer')
assert len(df_merged_l) == len(df_merged_o), (
       f"left join results should be eq to the outer join"
)
rc_explode_df = df_merged_l
rc_explode_df.head()

,symbol,action,lots,open_date,close_date,open_price,close_price,profits,pips,commission,...,profits_calc,commission_by_pips,date_var,swap_per_date_var_and_pip,open_date_mod_5,close_date_mod_5,dt_range,dt_point_holding,dt_range_inter_num,cur_holding_cnt
0,EURUSD,buy,0.01,2021-07-09 02:05:44,2021-07-09 03:08:27,1.18363,1.18375,0.12,1.2,-0.04,...,-0.028,-0.4,0,0.0,2021-07-09 02:05:00,2021-07-09 03:05:00,0 days 01:00:00,2021-07-09 02:05:00,13,1
1,EURUSD,buy,0.01,2021-07-09 02:05:44,2021-07-09 03:08:27,1.18363,1.18375,0.12,1.2,-0.04,...,-0.028,-0.4,0,0.0,2021-07-09 02:05:00,2021-07-09 03:05:00,0 days 01:00:00,2021-07-09 02:10:00,13,1
2,EURUSD,buy,0.01,2021-07-09 02:05:44,2021-07-09 03:08:27,1.18363,1.18375,0.12,1.2,-0.04,...,-0.028,-0.4,0,0.0,2021-07-09 02:05:00,2021-07-09 03:05:00,0 days 01:00:00,2021-07-09 02:15:00,13,1
3,EURUSD,buy,0.01,2021-07-09 02:05:44,2021-07-09 03:08:27,1.18363,1.18375,0.12,1.2,-0.04,...,-0.028,-0.4,0,0.0,2021-07-09 02:05:00,2021-07-09 03:05:00,0 days 01:00:00,2021-07-09 02:20:00,13,1
4,EURUSD,buy,0.01,2021-07-09 02:05:44,2021-07-09 03:08:27,1.18363,1.18375,0.12,1.2,-0.04,...,-0.028,-0.4,0,0.0,2021-07-09 02:05:00,2021-07-09 03:05:00,0 days 01:00:00,2021-07-09 02:25:00,13,1


In [22]:
%%time
# save the records data to disk
rc_explode_df.set_index(keys=["dt_point_holding"], drop=False, inplace=True)
rc_explode_df.sort_index(axis=0, inplace=True)
rc_explode_df.to_csv("../data/yht_eur_big_table.csv")

CPU times: user 1.87 s, sys: 7.88 ms, total: 1.88 s
Wall time: 1.89 s
